## Integrantes:
1. Camila Coltriani
2. Luis Dartayet
3. Irania Fuentes
4. Jonathan Fichelson
5. Ornella Cevoli

# Trabajo práctico 2 : Modelo de regresión lineal del dataset Properatti

## Objetivos

El objetivo de este trabajo final es generar y comparar estadisticamente tres modelos de regresion lineal sobre el dataset limpio de Properatti construido en el TP_1; en este, fue planteado la hipotesis que el precio (variable objetivo) de las propiedades iba a estar influenciado principalmente por la superfice y la ubicacion (variables predictoras). 

Con base a esto, se han planteado los siguientes objetivos especificos:
- Explorar el dataset limpio con la finalidad de verificar si debe realizarse una ultima limpieza o pueden utilizase los datos directamente;
- Realizar una visualización general de las distribuciones y relaciones del dataset con la finalidad de determinar la zona, tipo de inmueble y variables predictoras y objetivo para la realizacion de los modelos;
- Construir modelos de regresión lineal simple y multiple e interpretar sus metricas con la finalidad de identificar el que mejor permita obtener una prediccion confiable de la variable objetivo;
- Implementar un modelo de regularización con la finalidad de compararlos y evaluar si existe o no problemas de sobreajuste;
- Determinar el modelo que más se ajusta al comportamiento de los datos analizados. 

In [ ]:
#Las librerías utilizadas en este documento son:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
sns.set()

In [ ]:
#ELIMINAR
# Con la finalidad de comparar el modelo en distintas Municipios:  La Matanza, Lomas de Zamora y Tigre 
#tienen más de 500 registros
#son de tres zonas distintas del conurbano
# la cantidad de registros no varia mucho entre ellos
# armar un modelo inicial con superficie total 
# ver el tema de la ubicacion relacionarla con algun punto y calcular distancias 
# variable target precio 
# variables predictoras superfice total y ubicacion #probar con otra: ambientes 
#armar tres datasets: uno por cada municipio 
#diferencias de precios en la misma zona



## Visualización del dataset

In [ ]:
# Visualización de la forma y atributos del dataset
data = pd.read_csv("./data/data_limpio_gdf.csv")
print(data.shape)
print("El dataset está compuesto por:", data.shape[0], "filas y",data.shape[1],"columnas.")
data.sample(5)

### Descripción de las columnas del dataset limpio:

Las columnas que incluye son:

● municipio: ubicacion del inmueble por su municipio/barrios

● provincia: ubicacion del inmueble por provincia

● lat  ●lon: ubicacion de latitud y longitud

● superficie_m2_total: superficie total en m² del inmueble

● price_usd: Precio en dolares del inmueble

● tipo: tipo de inmueble en venta (casa, departamento, ph, tienda)

● ambientes_cat: cantidad de ambientes del inmueble (0, 1, 2, 3 , 4 o más)

● precio_usd_por_m2: Precio en dolares por metro cuadrado (USD/m²: precio dólares / superficie)

● tipo_cat_code: categoria numerica de tipo de inmueble

● municipio_cat_code: categoria numerica de municipios

● provincia_cat_code: categoria numerica de provincia

● tipo_cat_code: categoria numerica de ambientes_cat

● geometry: figura geometrica de latitud y la longitud

● country_name: nombre del país donde ocurre la operacion inmobiliaría

● **precio_usd_por_m2_cat: categoria numerica de precio_usd_por_m2**

# Analisis exploratorio y visualizacion de correlaciones entre las variables

In [ ]:
#Revisamos la presencia de datos NaN
data.isna().sum().sort_values()
#La columna "ambientes_cat" quedó con 1248 registros nulos

In [ ]:
#reviso donde están ubicados y a que propiedad pertenecen los registros nulos para saber si afectaran escoger un tipo de inmueble y su zona
mascara_nulos = data["ambientes_cat"].astype(str) == "nan" 
data_nulos = data[mascara_nulos]
data_nulos.loc[:, ["municipio", 'tipo', 'ambientes_cat', "precio_usd"]].sample(7)
#print(data[mascara_nulos].index)

In [ ]:
#agrupamos los registros donde hay nulos
pd.options.display.max_rows = None
data_nulos.groupby(["tipo"])["municipio"].value_counts().sort_values(ascending=False)
#vemos que los nan están distribuidos equitativamente y no están concentrados en una mismo municipio

In [ ]:
#Los elimino p
data.dropna(subset=['ambientes_cat'], inplace=True)
print(data.isna().sum())

In [ ]:
#Realizamos una descripcion estadistica del dataset
data.describe()
#existen datos que no permiten ver los estadisticos ya que hay valores de 0 en sup_m2_total e inf en precio_usd_por_m2: eliminarlos

In [ ]:
#eliminamos del dataset los registros de sup_m2_total con valores de cero
data.drop(data[(data["sup_m2_total"] ==0)].index, inplace=True ,axis=0)

# Registros con valor cero en sup_m2_total
# row_ceros = data[(data["sup_m2_total"] ==0)].index
# data = data.drop(index=row_ceros)

# #import math
# #data[(data["precio_usd_por_m2"] == math.inf)].index
#son los mismos registros que para sup_m2_total

In [ ]:
data.describe()

In [ ]:
#Vemos la correlacion entre las variables 
data_corr = data.corr()
#graficamos
plt.figure(figsize=(6,6))
sns.heatmap(data_corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlación entre variables")
plt.show()

#puede verse una correlacion significativa entre sup_m2_total y precio_usd (0.39)
#tipo_cat_code / ambientes_cat_code y  precio_usd (0.30)
#precio_usd_por_m2_cat y lon

In [ ]:
#graficamos las provincias y municipios que contengan un valor minimo de 500 registros por municipio (para una mejor vsisualizacion)
limite = 500
data = data.copy().groupby(['municipio']).filter(lambda grp: grp.shape[0] > limite)

In [ ]:
fig= plt.subplots(figsize=(20,20),constrained_layout=True)
grid = gridspec.GridSpec(2, 1, height_ratios=[1, 3])

ax1=plt.subplot(grid[0])
sns.countplot(data=data,y="provincia",order=data["provincia"].value_counts().index ,ax=ax1,color="g")

ax1.set_yticklabels(ax1.get_yticklabels(),fontsize="medium")
ax1.set_title("Distribucion de registros segun la provincia", fontsize= 'large')

ax2=plt.subplot(grid[1])
sns.countplot(data=data,x="municipio",order=data["municipio"].value_counts().index,ax=ax2,color="b")


ax2.set_title("Distribucion de registros segun los municipios", fontsize= 'large')
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=90,ha="right")
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
ax1.grid()
ax2.grid()
plt.show()

La mayor cantidad de registros están Capital Federal para los barrios de Palermo, Belgrano, Caballito..
Consideraremos Capital Federal para la evaluación de los modelos 

In [ ]:
#Revisamos la distribucion de registros por tipo de inmueble
plt.figure(figsize=(5,3))

plt.gca().yaxis.set_major_formatter(ScalarFormatter())
ax = sns.countplot(data = data, x = "tipo")
ax.set_xticklabels(ax.get_xticklabels(),rotation=40,ha="right")
plt.show()

#Apartamentos tiene la mayoria de los datos

Consideraremos apartamentos para la evaluacion de los modelos

In [ ]:
#Revisamos la ubicacion de los los tipos de inmueble que contienen la mayor cantidad de registros
pd.options.display.max_rows = None
data.groupby(["municipio"])["tipo"].value_counts().sort_values(ascending=False)

## Correlacion entre la variables del dataset

In [ ]:
figz= plt.figure()
mask_cols= ["tipo","sup_m2_total","precio_usd","precio_usd_por_m2", "ambientes_cat", "municipio_cat_code", "ambientes_cat_code","lat"]
graph=sns.pairplot(data[mask_cols],hue="tipo")
graph.fig.set_size_inches(20,10)
plt.grid()
plt.show()


In [ ]:
# data_2 = data[["tipo","sup_m2_total","precio_usd","precio_usd_por_m2", "ambientes_cat", "municipio_cat_code", "ambientes_cat_code","lat"]]
# # g = sns.PairGrid(data_2, hue="tipo")
# g.map(sns.scatterplot)
# g.add_legend()

In [ ]:
# Realizamos los siguientes graficos para visualizar mejor (zoom) las relaciones
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "sup_m2_total", "precio_usd", alpha=.5)
g.add_legend()

#

In [ ]:
# data= data[(data["precio_usd"] >100) & (data["precio_usd"]<= 4.500000e+05)]
# data= data[(data["precio_usd_por_m2"] > 100) & (data["precio_usd_por_m2"]<= 2.500000e+04)]
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "sup_m2_total", "precio_usd_por_m2", alpha=.5)
g.add_legend()

In [ ]:
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "precio_usd_por_m2", "precio_usd", alpha=.7)
g.add_legend()

In [ ]:
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "municipio_cat_code", "precio_usd", alpha=.7)
g.add_legend()

In [ ]:
g = sns.FacetGrid(data, col="tipo")
g.map(sns.scatterplot, "ambientes_cat_code", "precio_usd", alpha=.7)
g.add_legend()

In [ ]:
# masc_dep = data[data["tipo"]=="apartment"]
# sns.scatterplot(data=masc_dep, x='sup_m2_total', y='precio_usd', palette="deep")
# plt.grid()
# plt.show()

In [ ]:
# manf_sort = pd.get_dummies(data['ambientes_cat_code']).sum().sort_values(ascending = False)
# plt.figure(figsize=(5,5))
# sns.barplot(x = manf_sort.index, y = manf_sort.values)
# plt.show()

In [ ]:
# manf_sort = pd.get_dummies(data['ambientes_cat']).sum().sort_values(ascending = False)
# plt.figure(figsize=(5,5))
# sns.barplot(x = manf_sort.index, y = manf_sort.values)

# plt.show()

In [ ]:
#masc_barrio = data[data["municipio"]=="Almagro"]
#sns.scatterplot(data=masc_barrio, x='sup_m2_total', y='precio_usd', palette="deep")
#plt.grid()
#plt.show()

In [ ]:
# masc_barrio = data[data["municipio"]=="Almagro"]
# sns.set_style('darkgrid')
# plt.figure(figsize=(6, 4))
# sns.boxplot(data=masc_barrio, x= "tipo", y="sup_m2_total")

# plt.xlabel("tipo")
# plt.ylabel("sup_m2_total")
# plt.title("Superficie total por tipo de inmueble ")

In [ ]:
# masc_barrio = data[data["municipio"]=="Almagro"]
# sns.set_style('darkgrid')
# plt.figure(figsize=(6, 4))
# sns.boxplot(data=masc_barrio, x= "tipo", y="precio_usd_por_m2")

# plt.xlabel("tipo")
# plt.ylabel("precio_usd_por_m2")
# plt.title("Precio/m2 por tipo de inmueble ")

In [ ]:
# masc_barrio = data[data["municipio"]=="Lomas de Zamora"]
# sns.scatterplot(data=masc_barrio, x='sup_m2_total', y='precio_usd', palette="deep")
# plt.grid()
# plt.show()

In [ ]:
#mostramos un boxplot para los outliers por tipo de propiedad
data.boxplot(column= "precio_usd", by="tipo") 
plt.show()

In [ ]:
#la variable target debe tener una distribucion cercana a la normal

In [ ]:
#de acuerdo con los valores minimos y maximos realizamos un histograma para ver la distrubicón de datos de precio_usd
data= data[(data["precio_usd"] >10000) & (data["precio_usd"]<= 4.500000e+05)]
sns.histplot(data["precio_usd"], color = "orange", bins = 40)

In [ ]:
#de acuerdo con los valores minimos y maximos realizamos un histograma para ver la distrubicón de datos de superficie total
data= data[(data["sup_m2_total"] >10) & (data["sup_m2_total"]<= 400)]
sns.histplot(data["sup_m2_total"], color = "green", bins = 40)

In [ ]:
data= data[(data["precio_usd_por_m2"] >10) & (data["precio_usd_por_m2"]<= 2.812500e+03)]
sns.histplot(data["precio_usd_por_m2"], color = "blue", bins = 20)


In [ ]:
data.boxplot(column= "precio_usd", by="tipo") 
plt.show()

In [ ]:

#sns.histplot(data['ambientes_cat'], bins = 5);

In [ ]:
#masc_barrio = data[data["municipio"]=="Tigre"]
#sns.scatterplot(data=masc_barrio, x='sup_m2_total', y='precio_usd', palette="deep")
#plt.grid()
#plt.show()

Conclusiones de la visualizacion de distribuciones:
- La mayor cantidad de registros están Capital Federal para los barrios de Palermo, Belgrano, Caballito..
    Consideraremos Capital Federal para la evaluación de los modelos 
- Utilizaremos departamentos como el tipo de inmueble a modelar por contener una mayor cantidad de datos

# Analisis inicial de regresiones sobre el dataset

## Regresión lineal simple

In [ ]:
#Iniciamos con un barrio de Capital Federal con una cantidad de registros mayor a 500 para ver como resulta el modelo
# data_Caballito = data[data['municipio'] == 'Caballito']
# data_Caballito.to_csv('./data/data_limpio_gdf_caballito.csv', index=False)

In [ ]:
df = pd.read_csv('./data/data_limpio_gdf_caballito.csv')

In [ ]:
# Asignamos la variable a predecir
y = df['precio_usd']
# Asignamos la variable predictora
X_simple = df[['sup_m2_total']]

# Dividimos el dataset en train y test

X_train, X_test, y_train, y_test = train_test_split(X_simple, y, test_size=0.2, random_state=42)

# Instanciamos el modelo y lo entrenamos

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
# Guardamos  las predicciones en un nuevo vector que llamaremos predictions.
predictions = lr.predict(X_simple)

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# Imprimimos el intercepto y los coeficientes como atributos del objeto entrenado.
print ('Intercepto=', ' ', lr.intercept_)
print ('sup_m2_total=', ' ', lr.coef_)
# imprimos la metrica que mide la bondad de ajusto del modelo. En este caso el R2.
print ('R2_train=', ' ', lr.score(X_train, y_train))
print ('R2_test=', ' ', lr.score(X_test, y_test))
print ("EMC:", mean_squared_error(y, predictions))
print ("r_EMC:", np.sqrt(mean_squared_error(y, predictions)))

In [ ]:
# # Generamos una función que resume los coeficientes, el intercepto y el R2
# # "model" = objeto con el modelo
# # "X" = matrix de variables independientes

# def sum_mod(lr, X):
#     a = pd.DataFrame(lr.coef_ , X_simple.columns.values)
#     a = a.append(pd.DataFrame([lr.intercept_, lr.score(X_simple, y)], index=['Intecept','R2']))
#     return(a)

In [ ]:
# Graficamos la variable X contra la variable Y
plt.scatter(X_simple, y, s=30, c='black', marker='+', zorder=10)
plt.scatter(X_simple, y)
plt.xlabel("sup_m2_total")
plt.ylabel("Valores reales precio_usd_por_m2")
plt.title('Relación entre sup_m2_total y precio_usd_por_m2')
plt.show()

# Graficamos el modelo
plt.plot(y,y, '-.',c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones de precio_usd_por_m2 usando sup_m2_total")
plt.ylabel("Valores reales precio_usd_por_m2")
plt.title('Comparación entre el modelo y los valores reales de precio_usd_por_m2')
plt.show()

In [ ]:
#Solo para departamentos

In [ ]:
tipo_dept = ["apartment"]
df_2 = df[df["tipo"].isin(tipo_dept)]

In [ ]:
# Asignamos la variable a predecir
y = df_2['precio_usd']

In [ ]:
# Asignamos la variable predictora
X_2 = df_2[['sup_m2_total']]

In [ ]:
# Dividimos el dataset en train y test

X_train, X_test, y_train, y_test = train_test_split(X_2, y, test_size=0.2, random_state=42)

In [ ]:
# Instanciamos el modelo y lo entrenamos

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
# Guardamos  las predicciones en un nuevo vector que llamaremos predictions.
predictions = lr.predict(X_2)

In [ ]:
# Imprimimos el intercepto y los coeficientes como atributos del objeto entrenado.
print ('Intercepto=', ' ', lr.intercept_)
print ('sup_m2_total=', ' ', lr.coef_)
# imprimos la metrica que mide la bondad de ajusto del modelo. En este caso el R2.
print ('R2_train=', ' ', lr.score(X, y))
print ('R2_train=', ' ', lr.score(X_train, y_train))
print ('R2_test=', ' ', lr.score(X_test, y_test))
print ("EMC:", mean_squared_error(y, predictions))
print ("r_EMC:", np.sqrt(mean_squared_error(y, predictions)))

In [ ]:
# Graficamos la variable X contra la variable Y
plt.scatter(X, y, s=30, c='black', marker='+', zorder=10)
plt.scatter(X, y)
plt.xlabel("sup_m2_total")
plt.ylabel("Valores reales precio_usd_por_m2")
plt.title('Relación entre sup_m2_total y precio_usd_por_m2')
plt.show()

# Graficamos el modelo
plt.plot(y,y, '-.',c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones de precio_usd_por_m2 usando sup_m2_total")
plt.ylabel("Valores reales precio_usd_por_m2")
plt.title('Comparación entre el modelo y los valores reales de precio_usd_por_m2')
plt.show()

In [ ]:
#revisamos la estadisticos de media, desviacion y rangos intercuartilicos
#df_2[["precio_usd_por_m2", "sup_m2_total" ]].describe()

In [ ]:
#vamos a verificar cuales son los datos que se encuentran por encima del tercer quartil y buscamos los limites superiores e inferiores
#para determinar que registros pueden considerarse extremos
#q1, q2, q3 = df_2["sup_m2_total"].quantile([0.25, 0.50, 0.75])
#print("Q1:", q1,"\nq2:", q2,"\nq3:", q3 )

In [ ]:
#para surface_total_in_m2 hayamos el limite superior: aplicamos la formula de calcular el rango intercuartilico y luego el valor superior
#iqr = q3-q1
#ls = q3 + 1.5*iqr
#print("El limite superior para considerar un outlier en superficie_m2_total es:", ls)

In [ ]:
# li = q1 - 1.5*iqr
# print("El limite inferior para considerar un outlier en superficie_m2_total es:", li)

In [ ]:
#aplicamos el metodo query para consultar que campos son los que cumplen con tener valores por encima del limite superior 
#datos_outliers_ls = 
#df_2.query("sup_m2_total > @ls").sample(4)
#vemos que estos valores extremos pertenecen a casas en zonas rurales por lo cuall tiene sentido sus superficie


In [ ]:
#limite_superior = 120
#masc_sup_m2_limite = df_2["sup_m2_total"] < limite_superior
#df_2.loc[masc_sup_m2_limite].head(4)

In [ ]:
#df_3 = df_2.loc[masc_sup_m2_limite]

In [ ]:
#df_3[["precio_usd_por_m2", "sup_m2_total" ]].describe()

In [ ]:
# Asignamos la variable a predecir
#y = df_3['precio_usd_por_m2']

In [ ]:
# Asignamos la variable predictora
#X = df_3[['sup_m2_total']]

In [ ]:
# Dividimos el dataset en train y test

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Instanciamos el modelo y lo entrenamos

#lr = linear_model.LinearRegression()
#lr.fit(X_train, y_train)

In [ ]:
# Guardamos  las predicciones en un nuevo vector que llamaremos predictions.
#predictions = lr.predict(X)

In [ ]:
# Imprimimos el intercepto y los coeficientes como atributos del objeto entrenado.
#print ('Intercepto=', ' ', lr.intercept_)
#print ('sup_m2_total=', ' ', lr.coef_)
# imprimos la metrica que mide la bondad de ajusto del modelo. En este caso el R2.
#print ('R2_train=', ' ', lr.score(X, y))

## Regresión lineal múltiple

In [ ]:
# Asignamos las variables predictoras

X_3 = df_2[['lat', 'lon', 'sup_m2_total']]

y = df_2['precio_usd']


In [ ]:
# Normalizamos los datos

scaler = StandardScaler()
scaler.fit(X)
X_3 = scaler.transform(X)


In [ ]:
# Dividimos en train y test

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_3, y, test_size=0.2, random_state=42)

In [ ]:
# Instanciamos el modelo y lo entrenamos

lr_2= linear_model.LinearRegression()
lr_2.fit(X_train_2, y_train_2)


In [ ]:
# Vemos los coeficientes

print('Coeficientes: ', lr_2.coef_)
print('Intercepto: ', lr_2.intercept_)

In [ ]:
# Calculamos el R2

print('R2: ', r2_score(y_test_2, lr_2.predict(X_test_2)))


In [ ]:
# Modelamos con statsmodels

X_train_sm = sm.add_constant(X_train_2)
model = sm.OLS(y_train_2, X_train_sm).fit()

print(model.summary())

In [ ]:
# Probamos con regularización ridge

lm_ridge = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))

model_ridge = lm_ridge.fit(X_train_2, y_train_2)

print(lm_ridge.alpha_)
print(lm_ridge.coef_)
print(lm_ridge.intercept_)

In [ ]:
model_ridge.score(X_test_2, y_test_2)

In [ ]:
# Probamos con regularización lasso

lm_lasso = linear_model.LassoCV(alphas=np.logspace(-6, 6, 13), cv=5)

model_lasso = lm_lasso.fit(X_train_2, y_train_2)

print(lm_lasso.alpha_)
print(lm_lasso.coef_)

print(lm_lasso.intercept_)

In [ ]:
model_lasso.score(X_test_2, y_test_2)

In [ ]:
dummy = pd.get_dummies(df_2['ambientes_cat'], drop_first=True)
dummy.head()

In [ ]:
df_3 = pd.concat([df_2["sup_m2_total"],dummy],axis=1)
df_3.head(4)

In [ ]:
# Asignamos las variables predictoras

X = df_3


y = df_2['precio_usd']


In [ ]:
# Normalizamos los datos

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


In [ ]:
# Dividimos en train y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Instanciamos el modelo y lo entrenamos

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)


In [ ]:
# Vemos los coeficientes

print('Coeficientes: ', lr.coef_)
print('Intercepto: ', lr.intercept_)

In [ ]:
# Calculamos el R2

print('R2: ', r2_score(y_test, lr.predict(X_test)))


In [ ]:
# Modelamos con statsmodels

X_train_sm = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_sm).fit()

model.summary()

In [ ]:
# Probamos con regularización ridge

lm_ridge = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))

model_ridge = lm_ridge.fit(X_train, y_train)

lm_ridge.alpha_

In [ ]:
model_ridge.score(X_test, y_test)

In [ ]:
# Probamos con regularización lasso

lm_lasso = linear_model.LassoCV(alphas=np.logspace(-6, 6, 13), cv=5)

model_lasso = lm_lasso.fit(X_train, y_train)

lm_lasso.alpha_

In [ ]:
model_lasso.score(X_test, y_test)

In [ ]:
# superficie_min=15
# superficie_max=1000

# data = data[(data.sup_m2_total <= 1000) & (data.sup_m2_total >= 15)]

# data = data[(data.precio_usd <= 4000000)]